In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from datetime import datetime

class HeroVRPTSolver:
    def __init__(self, heroes_file, objects_file, dist_start_file, dist_objects_file):
        self.heroes = pd.read_csv(heroes_file)
        self.objects = pd.read_csv(objects_file)
        self.dist_start = pd.read_csv(dist_start_file)
        self.dist_objects = pd.read_csv(dist_objects_file)
        self.visit_cost = 100
        self.hero_cost = 2500
        
    def preprocess_distances(self):
        """Создание полной матрицы расстояний"""
        n_objects = len(self.objects)
        self.full_dist_matrix = np.zeros((n_objects + 1, n_objects + 1))
        
        # Расстояния от замка (индекс 0) до объектов
        for idx, row in self.dist_start.iterrows():
            obj_id = row['object_id']
            self.full_dist_matrix[0, obj_id] = row['dist_start']
            self.full_dist_matrix[obj_id, 0] = row['dist_start']
        
        # Расстояния между объектами
        obj_ids = self.objects['object_id'].values
        for i, obj_i in enumerate(obj_ids):
            for j, obj_j in enumerate(obj_ids):
                if i != j:
                    self.full_dist_matrix[obj_i, obj_j] = self.dist_objects.iloc[i, j]

In [ ]:
data_path = '/kaggle/input/datafusiontask3-heroes/'
heroes_path = data_path+'data_heroes.csv'
objects_path = data_path+'data_objects.csv'
dist_objects_path = data_path+'dist_objects.csv'
dist_start_path = data_path+'dist_satrt.csv'

In [ ]:
solver = HeroVRPTSolver(
    heroes_file=heroes_path, 
    objects_file=objects_path, 
    dist_start_file=dist_objects_path, 
    dist_objects_file=dist_start_path
)

In [ ]:
def cluster_by_day(self):
    """Группировка объектов по дням открытия"""
    self.day_clusters = {}
    for day in range(1, 8):
        day_objects = self.objects[self.objects['day_open'] == day].copy()
        # Добавляем расстояние от замка
        day_objects = day_objects.merge(
            self.dist_start, on='object_id', how='left'
        )
        self.day_clusters[day] = day_objects.sort_values('dist_start')
    
    return self.day_clusters

In [ ]:
def greedy_initial_solution(self):
    """
    Построение начального решения жадным алгоритмом
    """
    routes = {hero_id: [] for hero_id in self.heroes['hero_id']}
    available_objects = set(self.objects['object_id'].values)
    
    # Сортируем героев по убыванию очков хода
    sorted_heroes = self.heroes.sort_values('move_points', ascending=False)
    
    for _, hero in sorted_heroes.iterrows():
        hero_id = hero['hero_id']
        current_pos = 0  # Замок
        current_day = 1
        remaining_points = hero['move_points']
        route = []
        
        while available_objects:
            # Находим доступные объекты с учетом дня
            feasible_objects = []
            
            for obj_id in available_objects:
                obj_info = self.objects[self.objects['object_id'] == obj_id].iloc[0]
                day_open = obj_info['day_open']
                
                # Проверяем, можем ли мы добраться с учетом времени
                dist = self.full_dist_matrix[current_pos, obj_id]
                
                # Оценка необходимых дней
                days_needed = self.estimate_days_needed(
                    dist, remaining_points, day_open, current_day
                )
                
                if days_needed is not None:
                    feasible_objects.append({
                        'object_id': obj_id,
                        'day_open': day_open,
                        'dist': dist,
                        'score': obj_info['reward'],
                        'days_needed': days_needed,
                        'heuristic': obj_info['reward'] / (dist + self.visit_cost)
                    })
            
            if not feasible_objects:
                break
            
            # Выбираем лучший объект по эвристике
            best_obj = max(feasible_objects, key=lambda x: x['heuristic'])
            
            # Добавляем в маршрут
            route.append(best_obj['object_id'])
            available_objects.remove(best_obj['object_id'])
            
            # Обновляем состояние
            current_pos = best_obj['object_id']
            current_day += best_obj['days_needed']
            remaining_points = hero['move_points']  # Сбрасываем для нового дня
            
            # Корректируем остаток с учетом правила последнего шага
            if best_obj['dist'] + self.visit_cost <= hero['move_points']:
                remaining_points -= (best_obj['dist'] + self.visit_cost)
        
        routes[hero_id] = route
        
        if not available_objects:
            break
    
    return routes

def estimate_days_needed(self, distance, remaining_points, day_open, current_day):
    """
    Оценка количества дней для достижения объекта
    """
    total_cost = distance + self.visit_cost
    
    if remaining_points >= total_cost:
        # Можем сделать в текущий день
        return 0 if current_day <= day_open else (current_day - day_open)
    else:
        # Нужен следующий день
        if current_day + 1 > day_open:
            return None  # Опоздаем
        return 1

In [ ]:
def local_search_improvement(self, routes):
    """
    Улучшение решения с помощью 2-opt и relocate операций
    """
    improved = True
    while improved:
        improved = False
        
        # 2-opt для каждого маршрута
        for hero_id, route in routes.items():
            if len(route) < 3:
                continue
                
            for i in range(len(route) - 2):
                for j in range(i + 2, len(route)):
                    new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
                    
                    if self.is_better_route(hero_id, new_route, route):
                        routes[hero_id] = new_route
                        improved = True
                        break
                if improved:
                    break
        
        # Перемещение объектов между героями
        if not improved:
            for hero1, hero2 in combinations(routes.keys(), 2):
                for obj in routes[hero1]:
                    if self.try_relocate(routes, hero1, hero2, obj):
                        improved = True
                        break
                if improved:
                    break
    
    return routes

def is_better_route(self, hero_id, new_route, old_route):
    """
    Проверка, лучше ли новый маршрут
    """
    new_score = self.calculate_route_score(hero_id, new_route)
    old_score = self.calculate_route_score(hero_id, old_route)
    return new_score > old_score

In [ ]:
def optimize_with_hiring_order(self, routes):
    """
    Оптимизация с учетом необходимости последовательного найма
    """
    # Определяем максимального использованного героя
    used_heroes = [hero_id for hero_id, route in routes.items() if route]
    max_hero_id = max(used_heroes) if used_heroes else 0
    
    # Перераспределяем маршруты между нанятыми героями
    if max_hero_id < 100:
        available_heroes = self.heroes[
            (self.heroes['hero_id'] <= max_hero_id) | 
            (self.heroes['hero_id'].isin(used_heroes))
        ]
        
        # Пытаемся улучшить распределение
        routes = self.redistribute_routes(routes, available_heroes)
    
    return routes, max_hero_id

def calculate_final_score(self, routes, max_hero_id):
    """
    Расчет финальной метрики
    """
    total_gold = 0
    
    for hero_id, route in routes.items():
        if not route:
            continue
            
        current_pos = 0
        current_day = 1
        remaining_points = self.heroes[
            self.heroes['hero_id'] == hero_id
        ]['move_points'].iloc[0]
        
        for obj_id in route:
            obj_info = self.objects[
                self.objects['object_id'] == obj_id
            ].iloc[0]
            
            dist = self.full_dist_matrix[current_pos, obj_id]
            
            # Симуляция посещения
            success, current_day, remaining_points = self.simulate_visit(
                obj_info, dist, current_day, remaining_points
            )
            
            if success:
                total_gold += obj_info['reward']
            
            current_pos = obj_id
    
    # Вычитаем стоимость найма
    total_gold -= max_hero_id * self.hero_cost
    
    return total_gold